In [1]:
import os
import requests
from dotenv import load_dotenv
from confluent_kafka import Consumer, KafkaException, KafkaError, Producer
import json

# Load environment variables from .env file
load_dotenv()

# Function to fetch stock data from Alpha Vantage API
def get_stock_data():
    api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey={api_key}'
    response = requests.get(url)
    return response.json()

# Configure Kafka consumer
def create_consumer():
    config = {
        'bootstrap.servers': os.getenv("KAFKA_BROKER"),
        'group.id': os.getenv("CONSUMER_GROUP"),
        'auto.offset.reset': 'earliest',
        'security.protocol': 'SASL_SSL',
        'sasl.mechanisms': 'PLAIN',
        'sasl.username': os.getenv("KAFKA_USERNAME"),
        'sasl.password': os.getenv("KAFKA_PASSWORD")
    }
    return Consumer(config)

# Configure Kafka producer
def create_producer():
    config = {
        'bootstrap.servers': os.getenv("KAFKA_BROKER"),
        'security.protocol': 'SASL_SSL',
        'sasl.mechanisms': 'PLAIN',
        'sasl.username': os.getenv("KAFKA_USERNAME"),
        'sasl.password': os.getenv("KAFKA_PASSWORD")
    }
    return Producer(config)

def main():
    consumer = create_consumer()
    producer = create_producer()
    topic = os.getenv("KAFKA_TOPIC")

    # Fetch and send data to Kafka
    stock_data = get_stock_data()
    if 'Time Series (5min)' in stock_data:
        for timestamp, values in stock_data['Time Series (5min)'].items():
            record_key = timestamp
            record_value = json.dumps(values)
            producer.produce(topic, key=record_key, value=record_value, callback=delivery_report)
        producer.flush()
        print("Data sent to Kafka topic.")

    consumer.subscribe([topic])

    try:
        while True:
            msg = consumer.poll(timeout=1.0)  # Adjust the timeout as needed

            if msg is None:
                continue

            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    print('%% %s [%d] reached end at offset %d\n' %
                          (msg.topic(), msg.partition(), msg.offset()))
                elif msg.error():
                    raise KafkaException(msg.error())
            else:
                print('Received message: {}'.format(msg.value().decode('utf-8')))

    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

if __name__ == '__main__':
    main()


Python-dotenv could not parse statement starting at line 16


Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [0]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_data [2]
Message delivered to stock_d